In [51]:
#import re
import sys, codecs, optparse, glob, os
import string, yaml, warnings
import random as rd
from ToolsMoodleXML2 import *
from cellbell import ding

In [52]:
fA1="Phrases-A1-mahira.csv"
fA2="Phrases-A2-mahira-comp.csv"
fA1B1="PhrasesA1B1.csv"
fA1B2="PhrasesA1B2.csv"
fA2B1="PhrasesA2B1.csv"
fA2B2="PhrasesA2B2.csv"
fB1="Phrases-B1-autruches.csv"
fB2="Phrases-B2.csv"
fB2comp="Phrases-B2-autruches-comp.csv"
fB2ext="Phrases-B2-extend.csv"

In [53]:
nKalaba="K3"
annee="21"
limiteDonnees=1500
freeSpace=False
dossier="/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/%s-%s"%(annee,nKalaba)
# enonces=["Forme-Traduction","Proprietes-Forme"]
# enonces=["Forme-Traduction","Traduction-Radical"]
# enonces=["Traduction-Radical"]
# enonces=["Proprietes-Forme","Glose-Decoupe"]
# enonces=["Glose-Decoupe"]
enonces=["Proprietes-Forme"]
achatAide=0 # 0 sans aide, 1 achat lettre, 2 achat mot
# fLettre="A1"
fLettre="B2"
theme="%s-%s-%s"%(annee,nKalaba,fLettre)

enonces=["Theme","Version"]
theme="Traductions"
freeSpace=True
achatAide=2 # 0 sans aide, 1 achat lettre, 2 achat mot
penalite=0.125
# enonces=["Version"]
# enonces=["Theme"]

# enonces=["Ordre"]
# theme="Syntagmes"

In [54]:
from os.path import expanduser
home = expanduser("~")
repertoire=home+dossier
base=repertoire+"/"

In [55]:
def chaine2utf8(chaine):
    if type(chaine)==str:
        result=unicode(chaine.decode('utf8'))
    elif type(chaine)==unicode:
        result=chaine
    return result

In [56]:
def substituerChamps(chaine, substituts):
    result=chaine
    for substitut in substituts:
        result=result.replace(substitut,substituts[substitut])
    return result

In [57]:
def preparationDonnees():
    for ligne in donneesLignes:
        if not (ligne.strip().strip(";")=="" or ligne.startswith("#")):
            temp=ligne.strip().split(";")
            result={"boucle":[temp],"conclusion":[]}
            titre=makeChamps(configuration[u"Titre"],temp).encode('utf8')
            donnees.append({titre:result})
    return

In [58]:
def numerotationDonnees(donnees):
    result={}
    for nDonnee, donnee in enumerate(donnees):
        result[u"#%d#"%(nDonnee+1)]=chaine2utf8(donnee)
    return result

In [59]:
def preparationConsigne(consigne,clozes,substituts):
    
    nbChamps=min(len(clozes),len(substituts))+1
    result=[]
    reponse=[]
    nbReponses=set()
    txtClozes=["#%d#"%n for n in range(1,nbChamps) if clozes["#%d#"%n]=="TXT"]
    if debug: print txtClozes
    repClozes=["#%d#"%n for n in range(1,nbChamps) if clozes["#%d#"%n] in ["SAC","SA"]]
    if debug: print repClozes
    if len(repClozes)>1:
        warnings.warn("Attention, pas de questions multiples avec une Short Answer RegExp")
    for element in consigne:
        chaine=chaine2utf8(element)
        for txtCloze in txtClozes:
#            print txtCloze,chaine
            chaine=chaine.replace(txtCloze,substituts[txtCloze])
        mList=re.findall(ur"#\d+#",chaine)
        nbReponses|=set(mList)
#        print mList,nbReponses
        if mList and len(nbReponses)==1:
            chaine=chaine.replace(mList[0],"")
            if chaine:
                result.append(chaine+ur"<br/>")
            reponse.append(substituts[mList[0]])
        else:
            result.append(chaine)
#        if chaine in clozes:
#            champ=substituerChamps(chaine,substituts)
#            if clozes[chaine]=="TXT":
#                result.append(champ)
#            else:
#                reponse.append(champ)
#        else:
#            result.append(chaine)
    return (result,reponse)

In [60]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in xrange(0, len(seq), size))

In [61]:
def makeQuestions(donnees):
    questions=XMLSeries(categorie)
    clozes=numerotationDonnees(boucleCLOZE)
    for exercices in donnees:
        for exercice in exercices:
            titre=chaine2utf8(exercice)
            substituts=numerotationDonnees(exercices[exercice]['boucle'][0])
            regexConsigne,regexReponse= preparationConsigne(consigne,clozes,substituts)
            regexConsigne=regexConsigne
            if freeSpace:
                freeSpaceReponses=[]
                for reponse in regexReponse:
                    newReponse=reponse.replace(u"'",u"['´’′❜]")
                    freeSpaceReponses.append(re.sub(r"\s+",r"\s+",newReponse))
                regexReponse.extend(freeSpaceReponses)
            questions.addExercice(RegExExercice(titre,[globalWrap[0]]+regexConsigne,regexReponse,usehint=achatAide,penalty=penalite))
    return questions

In [62]:
for enonce in enonces:
    yamlFichier=base+theme+"-"+enonce+".yaml"
    csvFichier=base+theme+".txt"
    boolEX=False
            
    with open(yamlFichier, 'r') as stream:
        configuration=yaml.safe_load(stream)

    with open(csvFichier, 'r') as file:
#         donneesLignes=rd.shuffle(file.readlines())
        donneesLignes=file.readlines()
    rd.shuffle(donneesLignes)
    
# Modifié pour produire des fichiers Moodle XML de limiteDonnees questions

    donnees=[]

    preparationDonnees()

        
    categorie=configuration[u"Catégorie"]+"/RX"
    consigne=[configuration[u"Boucle"][u"début"]]+configuration[u"Boucle"][u"contenu"]
    boucleCLOZE=configuration[u"Boucle"][u"Cloze"]
    globalWrap=[configuration[u"Décor"][u"fond"]+configuration[u"Décor"][u"entete"]+configuration[u"Global"][u"début"],configuration[u"Global"]["fin"]]
    for nC,chunk in enumerate(chunker(donnees,limiteDonnees)):
        questions=makeQuestions(chunk)
        xmlOutput=MoodleXML()
        xmlOutput.addQuiz(questions.getSeries())

        xmlFichier=yamlFichier.split(".")[0]+"-RX%d.xml"%nC

        with codecs.open(xmlFichier, 'w',encoding="utf8") as output:
            output.write(xmlOutput.getXML())

In [63]:
#print xmlFichier

In [64]:
#questionsConsigne.getConsigne(donnees)

In [65]:
#print xmlOutput.getXML()

In [66]:
# clozes

In [67]:
ding()

ImportError: Library "c" not found.